In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
%cd /content/drive/My Drive/Colab Notebooks/RoadWorksAnalyzer/data/

/content/drive/.shortcut-targets-by-id/1llXkQFhfIsRiSyYeqMtBZUykMH-b08ey/RoadWorksAnalyzer/data


In [5]:
import re
from datetime import datetime

import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df = pd.read_csv('Moscow_all_processed_data.csv', sep='^')
df.head()

,global_id,Address,AdmArea,District,WorksType,WorksStatus,Customer,ActualBeginDate,WorksBeginDate,ActualEndDate,PlannedEndDate,Lanes_closed,geoData,WorkYear
0,2639121078,"Фрунзенская наб., д.8",Центральный административный округ,район Хамовники,"Ремонтные работы, Работы ДЖКХ",идут,Unknown,2024-02-19,2024-02-19,2025-05-20,2025-05-20,1,"[[37.595787445, 55.732870129], [37.59622944, 5...",2025
1,2639494326,ул. Большая Академическая (ул. Валаамская - ул...,Северный административный округ,Тимирязевский район,"Ремонтные работы, Работы коммерческих организаций",закончены,Unknown,2024-01-15,2024-01-15,2024-12-08,2024-12-08,1,"[[37.560100034, 55.84447127], [37.560007415, 5...",2024
2,2639494348,ул. Новослободская (в сторону области) (ул. Па...,Центральный административный округ,Тверской район,"Ремонтные работы, Работы коммерческих организаций",закончены,Unknown,2024-05-29,2024-05-29,2024-11-19,2024-11-19,1,"[[37.597433083, 55.783436047], [37.597221136, ...",2024
3,2639494385,"ул. Маршала Тухачевского, д.15 - д.14, к.1",Северо-Западный административный округ,район Хорошёво-Мнёвники,"Ремонтные работы, Работы ДЖКХ",закончены,Unknown,2024-02-07,2024-02-07,2024-06-30,2024-06-30,1,"[[37.486243397, 55.781979222], [37.48678118, 5...",2024
4,2639494387,"Зелёный пр-кт, д.76",Восточный административный округ,район Новогиреево,"Ремонтные работы, Работы ДЖКХ",закончены,Unknown,2023-10-19,2023-10-19,2024-11-30,2024-11-30,1,"[[37.829881509, 55.74968833], [37.830577344, 5...",2024


Проверка на неполные данные

In [7]:
def incomplete_data(row):
    ness_columns = ['global_id', 'Address', 'AdmArea', 'District', 'WorksType', 'WorksStatus',
               'Customer','ActualBeginDate','WorksBeginDate', 'ActualEndDate',
               'PlannedEndDate', 'Lanes_closed','geoData', 'WorkYear']
    errors = set()
    exs_columns = row.index.tolist()
    for col in ness_columns:
        if col not in exs_columns:
            errors.add(col)
    if len(errors) == 0:
        return None
    return "INCOMPLETE_DATA", errors

In [8]:
ok = df.iloc[0].copy()
not_ok = ok.copy()
not_ok = not_ok.drop(labels=['ActualBeginDate'])

print("Ok", incomplete_data(ok))
print("Not ok", incomplete_data(not_ok))

Ok None
Not ok ('INCOMPLETE_DATA', {'ActualBeginDate'})


Проверка на неполные данные 2

In [9]:
def null_data(row):
    ness_columns = ['global_id', 'Address', 'AdmArea', 'District', 'WorksType', 'WorksStatus',
               'Customer','ActualBeginDate','WorksBeginDate', 'ActualEndDate',
               'PlannedEndDate', 'Lanes_closed','geoData', 'WorkYear']
    row = row.copy()

    for column in ness_columns:
        if column not in row.index:
            row[column] = np.nan

    nan_indices = row.index[row.isna()].tolist()
    if len(nan_indices) == 0:
        return None
    return "NULL_DATA", len(nan_indices), set(nan_indices)

In [10]:
ok = df.iloc[0].copy()
not_ok = ok.copy()
not_ok['global_id'] = np.nan
not_ok = not_ok.drop(labels=['ActualBeginDate'])

print("Ok", null_data(ok))
print("Not ok", null_data(not_ok))

Ok None
Not ok ('NULL_DATA', 2, {'global_id', 'ActualBeginDate'})


Проверка формата даты:

In [11]:
from datetime import datetime

def check_date_format_string(date_string):
    try:
        date_obj = datetime.strptime(str(date_string), '%Y-%m-%d')
        return True
    except ValueError:
        return False

def date_format(row):
    date_columns = ['ActualBeginDate', 'WorksBeginDate', 'ActualEndDate', 'PlannedEndDate']
    errors = set()
    for col in date_columns:
        if col not in row.index or not check_date_format_string(row[col]):
            errors.add(col)
    if len(errors) == 0:
        return None
    return "DATE_FORMAT_ERROR", errors

In [12]:
ok = df.iloc[0].copy()
not_ok = ok.copy()
not_ok['ActualBeginDate'] = '2024-12'
not_ok = not_ok.drop(labels=['WorksBeginDate'])

print("Ok", date_format(ok))
print("Not ok", date_format(not_ok))

Ok None
Not ok ('DATE_FORMAT_ERROR', {'WorksBeginDate', 'ActualBeginDate'})


Проверка формата координат

In [13]:
def coordinates_format(row):
    try:
        coords_text = row['geoData']
        for pair in coords_text.split("], ["):
            clean_pair = pair.replace("[", "").replace("]", "").strip()
            tmp = [float(coord) for coord in clean_pair.split(",")]
            lat, lon = tmp[1], tmp[0]
            if not ((0 <= abs(lat) <= 90) and (0 <= abs(lon) <= 180)):
                raise Exception()
        return None
    except:
        return 'DATE_COORDINATES_ERROR'

In [14]:
ok = df.iloc[0].copy()
not_ok = ok.copy()
print("Ok", coordinates_format(ok))

not_ok['geoData'] = '[[37.595787445, 55.732870129], [37.59622944, 155.733289645]]'
print("Not ok", coordinates_format(not_ok))

not_ok['geoData'] = '[[37.595787445, 55.732870129], [37.59622944, aaa]]'
print("Not ok", coordinates_format(not_ok))

Ok None
Not ok DATE_COORDINATES_ERROR
Not ok DATE_COORDINATES_ERROR


Проверка согласованности дат

In [15]:
def date_consistency(row):
    date_columns = ['ActualBeginDate', 'WorksBeginDate', 'ActualEndDate', 'PlannedEndDate']
    errors = set()
    if not ('ActualBeginDate' in row.index and 'ActualEndDate' in row.index and
        check_date_format_string(row['ActualBeginDate']) and check_date_format_string(row['ActualEndDate']) and
       row['ActualBeginDate'] <= row['ActualEndDate']):
        errors.add('ActualBeginDate & ActualEndDate')
    if not ('WorksBeginDate' in row.index and 'PlannedEndDate' in row.index and
        check_date_format_string(row['WorksBeginDate']) and check_date_format_string(row['PlannedEndDate']) and
       row['WorksBeginDate'] <= row['PlannedEndDate']):
        errors.add('WorksBeginDate & PlannedEndDate')
    if len(errors) == 0:
        return None
    return "DATE_CONSISTENCY", errors


In [16]:
ok = df.iloc[0].copy()
not_ok = ok.copy()
not_ok['ActualBeginDate'] = '2024-12'
not_ok = not_ok.drop(labels=['WorksBeginDate'])

print("Ok", date_consistency(ok))
print("Not ok", date_consistency(not_ok))

Ok None
Not ok ('DATE_CONSISTENCY', {'WorksBeginDate & PlannedEndDate', 'ActualBeginDate & ActualEndDate'})


Проверка на дубли

In [17]:
def data_duplicates(df):
    res = df.duplicated().sum()
    return 'DATA_DUPLICATES', res

In [18]:
print(data_duplicates(df))

new_df = pd.DataFrame([df.iloc[0], df.iloc[0]])
new_df.reset_index(drop=True, inplace=True)
print(data_duplicates(new_df))

('DATA_DUPLICATES', 0)
('DATA_DUPLICATES', 1)


Получим отчет по ошибкам по строкам

In [19]:
check_for_row = [null_data, coordinates_format, date_format, date_consistency]
error_df = df.copy()
error_df['NULL_DATA'] = 0 # метка, есть ли пропуски
error_df['NULL_DATA_COUNT'] = 0 # колво пустых колонк
error_df['DATE_COORDINATES_ERROR'] = 0
error_df['DATE_FORMAT_ERROR'] = 0
error_df['DATE_CONSISTENCY'] = 0
for idx, row in error_df.iterrows():
    for f in check_for_row:
        res = f(row)
        if res == None:
            continue
        error, args = res[0], res[1:]
        error_df.loc[idx, error] = 1
        if error == 'NULL_DATA':
            error_df.loc[idx, 'NULL_DATA_COUNT'] = res[1]
total = df.shape[0]
errors_rows = {
    'NULL_DATA': f'{error_df["NULL_DATA"].sum() / total * 100:.4f}%',
    'NULL_DATA_COUNT': f'{error_df["NULL_DATA_COUNT"].sum() / (total * len(df.columns)):.4f}%',
    'DATE_COORDINATES_ERROR': f'{error_df["DATE_COORDINATES_ERROR"].sum() / total * 100:.4f}%',
    'DATE_FORMAT_ERROR': f'{error_df["DATE_FORMAT_ERROR"].sum() / total * 100:.4f}%',
    'DATE_CONSISTENCY': f'{error_df["DATE_CONSISTENCY"].sum() / total * 100:.4f}%',
}
errors_rows

{'NULL_DATA': '5.2043%',
 'NULL_DATA_COUNT': '0.0049%',
 'DATE_COORDINATES_ERROR': '0.0000%',
 'DATE_FORMAT_ERROR': '5.2043%',
 'DATE_CONSISTENCY': '5.2043%'}

Получим  отчет по ошибкам по всему датасету

In [20]:
check_for_df = [data_duplicates]
errors_df = {}
for f in check_for_df:
    res = f(df)
    errors_df[res[0]] = f'{res[1] / total * 100:.4f}%'
errors_df

{'DATA_DUPLICATES': '0.0000%'}

Объединим

In [21]:
errors = {**errors_rows, **errors_df}
errors

{'NULL_DATA': '5.2043%',
 'NULL_DATA_COUNT': '0.0049%',
 'DATE_COORDINATES_ERROR': '0.0000%',
 'DATE_FORMAT_ERROR': '5.2043%',
 'DATE_CONSISTENCY': '5.2043%',
 'DATA_DUPLICATES': '0.0000%'}

Все вместе

In [22]:
def quality(df):
    total = df.shape[0]
    check_for_row = [null_data, coordinates_format, date_format, date_consistency]

    error_df = df.copy()
    error_df['NULL_DATA'] = 0 # метка, есть ли пропуски
    error_df['NULL_DATA_COUNT'] = 0 # колво пустых колонк
    error_df['DATE_COORDINATES_ERROR'] = 0
    error_df['DATE_FORMAT_ERROR'] = 0
    error_df['DATE_CONSISTENCY'] = 0
    for idx, row in error_df.iterrows():
        for f in check_for_row:
            res = f(row)
            if res == None:
                continue
            error, args = res[0], res[1:]
            error_df.loc[idx, error] = 1
            if error == 'NULL_DATA':
                error_df.loc[idx, 'NULL_DATA_COUNT'] = res[1]
    errors_rows = {
        'NULL_DATA': f'{error_df["NULL_DATA"].sum() / total * 100:.4f}%',
        'NULL_DATA_COUNT': f'{error_df["NULL_DATA_COUNT"].sum() / (total * len(df.columns)):.4f}%',
        'DATE_COORDINATES_ERROR': f'{error_df["DATE_COORDINATES_ERROR"].sum() / total * 100:.4f}%',
        'DATE_FORMAT_ERROR': f'{error_df["DATE_FORMAT_ERROR"].sum() / total * 100:.4f}%',
        'DATE_CONSISTENCY': f'{error_df["DATE_CONSISTENCY"].sum() / total * 100:.4f}%',
    }

    check_for_df = [data_duplicates]
    errors_df = {}
    for f in check_for_df:
        res = f(df)
        errors_df[res[0]] = f'{res[1] / total * 100:.4f}%'
    return {**errors_rows, **errors_df}

In [23]:
quality(df)

{'NULL_DATA': '5.2043%',
 'NULL_DATA_COUNT': '0.0049%',
 'DATE_COORDINATES_ERROR': '0.0000%',
 'DATE_FORMAT_ERROR': '5.2043%',
 'DATE_CONSISTENCY': '5.2043%',
 'DATA_DUPLICATES': '0.0000%'}